In [1]:
import numpy as np


In [2]:
x_train = np.array([[1,1],[1,2],[1,3],[1,4]])
y_train = np.array([[1],[2],[6],[5]])
q_c = np.array([0.5,0.5,0.5,0.5])
siq = 1
yc_1 = np.array([-1,2])
yc_2 = np.array([1,-2])
yc = np.array([[[-1,2]],[[1,-2]]])
beta = np.array([[0,0]])

# mixture of experts using the dummy datasets above

In [3]:
def p_y_xc(target,betas, data,sigma,):
    top = np.exp((np.subtract(target, np.dot(data,np.transpose(betas))))**2)
    top = -(top / (2 * (sigma**2)))
    bottom = np.sqrt(2*3.14*(sigma**2))
    final = top/bottom
    return final

def s_c_x(step,data,):
    cont = []
    for i in range(len(step)):
        top = np.exp(np.dot(data,np.transpose(step[i])))
        bottom1 = np.exp(np.dot(data,np.transpose(step[i]))) 
        if i == 0:
            bottom2 = np.exp(np.dot(data,np.transpose(step[1])))
        elif i == 1 :
            bottom2 = np.exp(np.dot(data,np.transpose(step[0])))
        bottom = bottom1+bottom2
        final = top/bottom
        cont.append(final)
    a = cont[0].flatten().tolist()
    b = cont[1].flatten().tolist()
    s = np.array([a,b])
    s = s.reshape(2,4).transpose()
    return s

def stepc(data,qc,s_c):
    delta_yc = np.dot(np.transpose(data), (qc - s_c))
    return delta_yc
        

def beta_c(data, qc,target):
    top = np.dot(np.dot(data.transpose(),np.diag(qc)),target)
    bottom = np.dot(np.dot(data.transpose(),np.diag(qc)),data)
    betanew =np.dot(np.linalg.inv(bottom),top).reshape(1,2)
    return betanew

def qn_c(data,step,sigma,betas,target):
    cont = []
    for i in range(len(step)):
        yy = np.subtract(target, np.dot(data,np.transpose(betas)))**2
        top = np.exp(np.dot(data,step[i].transpose()) - ((1/(2* (sigma**2))) * yy))
        bottom1 = np.exp(np.dot(data,step[i].transpose()) - (1/(2* (sigma**2))) * yy)
        if i == 0:
            bottom2 = np.exp(np.dot(data,step[i].transpose()) - (1/(2* (sigma**2))) * yy)
        elif i == 1 :
            bottom2 = np.exp(np.dot(data,step[i].transpose()) - (1/(2* (sigma**2))) * yy)
        bottom = bottom1+bottom2
        final = top/bottom
        cont.append(final)
    a = cont[0].flatten().tolist()
    b = cont[1].flatten().tolist()
    s = np.array([a,b])
    s = s.reshape(2,4).transpose()
    return s

In [4]:
def mix_experts(data,target,sigma,step,betas,qc):
    softmax_gate = s_c_x(step,data,)
    print("softmax gating function for each model is\n {}".format(softmax_gate))
    
    minimized_yc = stepc(data,qc.reshape(4,1),softmax_gate)
    print("the minimized loss with respect to the step lenght for each model, optimal y_c =\n {}".format(minimized_yc))
    
    new_beta = beta_c(data, qc,target)
    print("for the first iteration both models have same betas which is beta_c:\n {}".format(new_beta))
    
    weights_q = qn_c(data,step,sigma,new_beta,target)
    print("the optimal value of weight qn_c for each model are \n{}".format(weights_q))
    
    return(new_beta)
    


# Since each model c is an expert in some region of predictor space, defined by its componet weight (gating function), the gating fuction serves the purpose of combining the predictions of each experts to generate an aggregated prediction of the mixture model.

# Results

In [5]:
model = mix_experts(x_train,y_train,siq,yc,beta,q_c)

softmax gating function for each model is
 [[8.80797078e-01 1.19202922e-01]
 [9.97527377e-01 2.47262316e-03]
 [9.99954602e-01 4.53978687e-05]
 [9.99999168e-01 8.31528028e-07]]
the minimized loss with respect to the step lenght for each model, optimal y_c =
 [[-1.87827823  1.87827823]
 [-4.87571231  4.87571231]]
for the first iteration both models have same betas which is beta_c:
 [[-0.5  1.6]]
the optimal value of weight qn_c for each model are 
[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]


# predictions

In [6]:
prediction = np.dot([1,2.5],model.transpose())
prediction

array([3.5])